In [17]:
import json
import time
import requests
import datetime
import telegram
import traceback
import pyupbit
from pytz import timezone
from decimal import *

## 함수

In [18]:
# 거래 대금 상위 n개
def get_top_k(n):
    coin_list = []
    tickers = pyupbit.get_tickers(fiat="KRW")
    market_code = ','.join(tickers)

    url = "https://api.upbit.com/v1/ticker"
    params = {
        "markets": market_code
    }
    response = requests.get(url, params=params).json()

    for info in response:
        coin_list.append([info['market'], info['acc_trade_price_24h']])

    coin_list.sort(key=lambda x: x[1], reverse=True)

    top_k_list = []
    for i in coin_list:
        top_k_list.append(i[0])
    return top_k_list[:n]


# 당일 최고가 계산 (프로그램 시작시 이미 목표가 돌파한 코인 구매 방지)
def get_highest_price(tickers):
    highest_price = dict()
    market_code = ','.join(tickers)
    url = "https://api.upbit.com/v1/ticker"
    params = {
        "markets": market_code
    }
    response = requests.get(url, params=params).json()

    for info in response:
        highest_price[info['market']] = info['high_price']

    return highest_price


# 목표가 계산
def get_target_price_list(tickers, k):
    target_price_list = dict()
    for ticker in tickers:
        target_price_list[ticker] = get_target_price(ticker, k)
        time.sleep(0.1)  # 요청 수 제한 (초당 10회)

    return target_price_list


def get_target_price(ticker, k):
    yesterday = datetime.date.today() - datetime.timedelta(1)
    df = pyupbit.get_ohlcv(ticker, interval="day", count=2)
    chart_yesterday = df.index[0].date()
    if chart_yesterday != yesterday:
        target_price = None
    else:
        target_price = df.iloc[0]['close'] + (df.iloc[0]['high'] - df.iloc[0]['low']) * k

    return target_price


# 이동평균선 계산
def get_ma(ticker, days):
    df = pyupbit.get_ohlcv(ticker, interval="day", count=days+1)
    ma = df['close'].rolling(window=days).mean()
    return ma[-2]


def get_ma_list(tickers, days):
    ma_list = dict()
    for ticker in tickers:
        ma_list[ticker] = get_ma(ticker, days)
        time.sleep(0.1)

    return ma_list



In [19]:
# Load API keys
with open("setting.json") as f:
    setting_loaded = json.loads(f.read())

# Upbit
access = setting_loaded["access_key"]
secret = setting_loaded["secret_key"]

# Telegram
telegram_token = setting_loaded["telegram_token"]
telegram_chat_id = setting_loaded["telegram_chat_id"]
telegram_bot = telegram.Bot(token=telegram_token)

# Initialize
upbit = pyupbit.Upbit(access, secret)
start_time = datetime.time(9)
sell_time = datetime.time(8, 59, 30)
tickrate = 0.5
fee = 0.0005
top_k = 20
portfolio_limit = 4
k_value = 0.15
stop_loss = 0.05
buymode = True  # ToDo: 매수 on off 추가
buy_time_data = {}
if __name__ == "__main__":
    fmt = "%Y-%m-%d %H:%M:%S"

In [20]:
def telegram_send(message):
    telegram_bot.sendMessage(chat_id=telegram_chat_id, text='[UPbit 자동매매]\n'+message)


# # Load saved data
# try:
#     with open("data.json") as f:
#         data_loaded = json.loads(f.read())
#
#     updated_at = datetime.datetime.strptime(data_loaded['updated_at'], '%Y-%m-%d %H:%M:%S')
#
# except:
#     first_time = True
#
# if first_time:
#     updated_at = datetime.datetime.now()
#     top_k_list = get_top_k(top_k)
#     highest_price = get_highest_price(top_k_list)
#     ma_list = get_ma_list(top_k_list, 5)
#     balance = upbit.get_balance(ticker="KRW")
#     hold = []

updated_at = datetime.datetime.now(timezone('Asia/Seoul'))
top_k_list = get_top_k(top_k)
target_price_list = get_target_price_list(top_k_list, k_value)
highest_price = get_highest_price(top_k_list)
ma_list = get_ma_list(top_k_list, 5)
balance = upbit.get_balance(ticker="KRW")
hold = []
highest_price = get_highest_price(top_k_list)
high_price_track = highest_price
buy_list = []
buy_price = dict()
first_day = True

telegram_send('📢 프로그램 시작')


In [21]:
#코인티커 순서 확인 문자 
a = 0

#구매여부 확인 문자
b = 0

#매수구매 확인 문자
c = 0

#구매가격 확인 문자
d = 0

#현제시세 확인 문자
e = 0

#수익률 확인 문자
h = 0


In [ ]:
while True:
    try:
        #코인 목록 수
        coin_num = len(pyupbit.get_tickers(fiat="KRW"))
        print(a)
        
        #코인이름 가져오기
        if a < coin_num: 
            tickers = pyupbit.get_tickers(fiat="KRW")[a]
        else:
            tickers = pyupbit.get_tickers(fiat="KRW")[0]


        #구매여부 확인 코드
        if b <= coin_num-1:
            buy_time_data.update({ tickers : True})
            b = b + 1
            
        elif b > coin_num-1:
            b = b + 1
        
        else:
            b = b + 1
            
        #매수 구매 가격 저장코드    
        if c <= coin_num-1:
            buy_price.update({ tickers : 0 })
            c = c + 1
            
        elif c > coin_num-1:
            c = c + 1
        
        else:
            c = c + 1
        now = datetime.datetime.now(timezone('Asia/Seoul'))
        current_price_list = pyupbit.get_current_price(top_k_list)
 

        #코인 현재시세+현재가격
        tickers_now_time_price = pyupbit.get_current_price(tickers), now.strftime(fmt)
        
        #코인 현재시세
        tickers_now_price2 = int(pyupbit.get_current_price(tickers))
        
        
        
        
        
        #코인 신호선(9분 이동평균)
        tickers_df1 = pyupbit.get_ohlcv(tickers, interval="minute1", count=9)
        tickers_signal = tickers_df1['close'].rolling(9).mean().iloc[-1]
        
        #코인 12분 이동평균
        tickers_df2 = pyupbit.get_ohlcv(tickers, interval="minute1", count=12)
        tickers_ma12 = tickers_df2['close'].rolling(12).mean().iloc[-1]
        
        #1분 전 코인 12분 이동평균
        tickers_df3 = pyupbit.get_ohlcv(tickers, interval="minute1", count=12)
        tickers_ma12_1mb = tickers_df3['close'].rolling(12).mean().iloc[-2]
        
        #코인 21분 이동평균
        tickers_df4 = pyupbit.get_ohlcv(tickers, interval="minute1", count=21)
        tickers_ma21 = tickers_df4['close'].rolling(21).mean().iloc[-1]
        
        #1분 전 코인 21분 이동평균
        tickers_df5 = pyupbit.get_ohlcv(tickers, interval="minute1", count=21)
        tickers_ma21_1mb = tickers_df5['close'].rolling(21).mean().iloc[-2]
        
        
        if a < coin_num-1:
             #매수조건 = 기본 매수조건
                if sell_time.hour == now.hour and sell_time.minute == now.minute and sell_time.second <= now.second < sell_time.second + 30:
                    while hold:
                        ticker = hold.pop()
                        amount = upbit.get_balance(ticker)
                        response = upbit.sell_market_order(ticker, amount)
                        income = (current_price_list[ticker] - buy_price[ticker]) * amount
                        telegram_send(f'⌛ 종가 매도\n📊 종목: {ticker}\n매도가: {current_price_list[ticker]}\n매수가: {buy_price[ticker]}\n수익: {income}\n{response}\n')
                    time.sleep(30)
                    continue
                    
                # 9시 이전 판매, 9시 이후 새 타겟 프라이스 갱신 구현
                if start_time.hour == now.hour and start_time.minute + 1 == now.minute and start_time.second <= now.second < start_time.second + 10:
                    updated_at = now
                    top_k_list = get_top_k(top_k)
                    target_price_list = get_target_price_list(top_k_list, k_value)
                    ma_list = get_ma_list(top_k_list, 5)
                    prev_balance = balance
                    balance = upbit.get_balance(ticker="KRW")
                    buy_list = []
                    buy_price = dict()
                    first_day = False
                    telegram_send(f'⏳ 9시 재설정\n전일 잔고: {prev_balance}\n금일 잔고: {balance}\n수익: {balance - prev_balance}')
                    time.sleep(10)
                    continue
                    
                for ticker in top_k_list:
                # 목표가 없을시 갱신하고 갱신해도 없을시 continue
                    if target_price_list[ticker] is None:
                        target_price_list[ticker] = get_target_price(ticker, k_value)
                        if target_price_list[ticker] is None:
                            continue


                 # 프로그램 가동 첫 날이면 고가와 목표가 비교 (이미 갱신했을 경우 구매 방지)
                    if first_day and highest_price[ticker] > target_price_list[ticker]:
                        continue
                        
                    # 목표가 돌파 및 5일 이평선 기준 상승장일시 매수
                    if a < coin_num-1:
                    #매수조건 = 기본 매수조건    
                        if  buy_time_data[tickers] == True and buy_price[tickers] == 0: 
                         #매수조건 = 9분 평균선이 MACD선(12분-21분) 을 상향돌파 즉,골든크로스 발생  
                            if  tickers_signal < ( tickers_ma12 - tickers_ma21 ):
                                if current_price_list[ticker] > target_price_list[ticker] and current_price_list[ticker] > ma_list[ticker] and len(buy_list) < portfolio_limit and buymode:
                                    response = upbit.buy_market_order(ticker, balance / portfolio_limit * (1-fee))
                                    high_price_track[ticker] = current_price_list[ticker]
                                    hold.append(ticker)
                                    buy_price[ticker] = current_price_list[ticker]
                                    buy_list.append(ticker)
                                    telegram_send(f'🛒 목표가 매수\n종목: {ticker}\n매수가: {current_price_list[ticker]}\n목표가: {target_price_list[ticker]}\n{response}')
                                    buy_time_data.update({ tickers : False })
                                    buy_price.update({ tickers : tickers_now_price2 })
                                    d = int(buy_price[tickers])
                                    a = a + 1
                            else:
                                print(tickers +" : 구매전 매수 준비중")
                                print(tickers_now_time_price)
                                a = a + 1
                        
                        elif  buy_time_data[tickers] == False and buy_price[tickers] > 0:
                            if ticker in buy_list:
                                if ticker in hold:
                                    high_price_track[ticker] = max(high_price_track[ticker], current_price_list[ticker])
                                    if  ((( tickers_now_price2 / d ) -1 ) * 100) < -1:
                                        if current_price_list[ticker] < high_price_track[ticker]*(1-stop_loss):
                                            amount = upbit.get_balance(ticker)
                                            response = upbit.sell_market_order(ticker, amount)
                                            hold.remove(ticker)
                                            income = (current_price_list[ticker] - buy_price[ticker]) * amount
                                            telegram_send(f'📉 매도\n종목: {ticker} / 매도가: {current_price_list[ticker]}\n매수가: {buy_price[ticker]}\n수익: {income}\n{response}')
                                            d = int(buy_price[tickers])
                                            e = int(tickers_now_price2)
                                            f = ((( e / d ) - 1 ) * 100 )
                                            print("수익률 :" , "%.2f" % (f) , "%")
                                            h = f + h
                                            print("수익률 합 : ","%.2f" % (h) , "%")
                                            telegram_send(f"📉수익률 : " + "%.2f" % (f) + "%" )
                                            telegram_send(f"📉총 수익률 : " + "%.2f" % (h) + "%" )
                                            buy_time_data.update({ tickers : True}) 
                                            buy_price.update({ tickers : 0 })
                                            a = a + 1
                                            
                                    if  ( tickers_ma12 - tickers_ma21 ) < (tickers_ma12_1mb - tickers_ma21_1mb) or tickers_signal > ( tickers_ma12 - tickers_ma21 ) :
                                            amount = upbit.get_balance(ticker)
                                            response = upbit.sell_market_order(ticker, amount)
                                            hold.remove(ticker)
                                            income = (current_price_list[ticker] - buy_price[ticker]) * amount
                                            telegram_send(f'📉 매도\n종목: {ticker} / 매도가: {current_price_list[ticker]}\n매수가: {buy_price[ticker]}\n수익: {income}\n{response}')
                                            d = int(buy_price[tickers])
                                            e = int(tickers_now_price2)
                                            f = ((( e / d ) - 1 ) * 100 )
                                            print("수익률 :" , "%.2f" % (f) , "%")
                                            h = f + h
                                            print("수익률 합 : ","%.2f" % (h) , "%")
                                            telegram_send(f"📉수익률 : " + "%.2f" % (f) + "%" )
                                            telegram_send(f"📉총 수익률 : " + "%.2f" % (h) + "%" )
                                            buy_time_data.update({ tickers : True}) 
                                            buy_price.update({ tickers : 0 })
                                            a = a + 1

                                    else:
                                        print(tickers +" : 구매후 매도 준비중")
                                        print(tickers_now_time_price)
                                        a = a + 1
                                            
                                            
                                            
        elif a >= coin_num - 1:
            a = 0
            
            
        else:
            a = 0
        time.sleep(0.5)
        
        
    except Exception as e:
        print("ERROR")
        post_message(myToken,"#bitcoin-stock", "에러발생" )
        time.sleep(0.5)

0
KRW-BTC : 구매전 매수 준비중
(70470000.0, '2021-10-14 15:44:33')
1
KRW-ETH : 구매전 매수 준비중
(4427000.0, '2021-10-14 15:44:34')
2
KRW-NEO : 구매전 매수 준비중
(54890.0, '2021-10-14 15:44:35')
3
KRW-MTL : 구매전 매수 준비중
(4265.0, '2021-10-14 15:44:36')
4
KRW-LTC : 구매전 매수 준비중
(218750.0, '2021-10-14 15:44:37')
5
KRW-XRP : 구매전 매수 준비중
(1375.0, '2021-10-14 15:44:38')
6
KRW-ETC : 구매전 매수 준비중
(65090.0, '2021-10-14 15:44:39')
7
KRW-OMG : 구매전 매수 준비중
(18010.0, '2021-10-14 15:44:40')
8
KRW-SNT : 구매전 매수 준비중
(113.0, '2021-10-14 15:44:41')
9
KRW-WAVES : 구매전 매수 준비중
(40240.0, '2021-10-14 15:44:41')
10
KRW-XEM : 구매전 매수 준비중
(215.0, '2021-10-14 15:44:42')
11
KRW-QTUM : 구매전 매수 준비중
(16170.0, '2021-10-14 15:44:43')
12
KRW-LSK : 구매전 매수 준비중
(4980.0, '2021-10-14 15:44:44')
13
KRW-STEEM : 구매전 매수 준비중
(766.0, '2021-10-14 15:44:45')
14
KRW-XLM : 구매전 매수 준비중
(460.0, '2021-10-14 15:44:46')
15
KRW-ARDR : 구매전 매수 준비중
(424.0, '2021-10-14 15:44:47')
16
KRW-ARK : 구매전 매수 준비중
(2680.0, '2021-10-14 15:44:48')
17
KRW-STORJ : 구매전 매수 준비중
(1620.0, '2021-10

46
KRW-BSV : 구매전 매수 준비중
(212000.0, '2021-10-14 15:46:56')
47
KRW-THETA : 구매전 매수 준비중
(7420.0, '2021-10-14 15:46:57')
48
KRW-QKC : 구매전 매수 준비중
(44.5, '2021-10-14 15:46:58')
49
KRW-BTT : 구매전 매수 준비중
(4.61, '2021-10-14 15:46:59')
50
KRW-MOC : 구매전 매수 준비중
(214.0, '2021-10-14 15:47:00')
51
KRW-ENJ : 구매전 매수 준비중
(1930.0, '2021-10-14 15:47:01')
52
KRW-TFUEL : 구매전 매수 준비중
(357.0, '2021-10-14 15:47:02')
53
KRW-MANA : 구매전 매수 준비중
(943.0, '2021-10-14 15:47:03')
54
KRW-ANKR : 구매전 매수 준비중
(108.0, '2021-10-14 15:47:04')
55
KRW-AERGO : 구매전 매수 준비중
(408.0, '2021-10-14 15:47:04')
56
KRW-ATOM : 구매전 매수 준비중
(41680.0, '2021-10-14 15:47:05')
57
KRW-TT : 구매전 매수 준비중
(18.2, '2021-10-14 15:47:06')
58
KRW-CRE : 구매전 매수 준비중
(14.8, '2021-10-14 15:47:07')
59
KRW-MBL : 구매전 매수 준비중
(13.9, '2021-10-14 15:47:08')
60
KRW-WAXP : 구매전 매수 준비중
(351.0, '2021-10-14 15:47:09')
61
KRW-HBAR : 구매전 매수 준비중
(443.0, '2021-10-14 15:47:10')
62
KRW-MED : 구매전 매수 준비중
(90.2, '2021-10-14 15:47:11')
63
KRW-MLK : 구매전 매수 준비중
(1435.0, '2021-10-14 15:47:12'

92
KRW-SAND : 구매전 매수 준비중
(903.0, '2021-10-14 15:49:24')
93
KRW-HUM : 구매전 매수 준비중
(249.0, '2021-10-14 15:49:25')
94
KRW-DOGE : 구매전 매수 준비중
(287.0, '2021-10-14 15:49:26')
95
KRW-STRK : 구매전 매수 준비중
(68030.0, '2021-10-14 15:49:27')
96
KRW-PUNDIX : 구매전 매수 준비중
(2040.0, '2021-10-14 15:49:28')
97
KRW-FLOW : 구매전 매수 준비중
(22730.0, '2021-10-14 15:49:29')
98
KRW-DAWN : 구매전 매수 준비중
(4225.0, '2021-10-14 15:49:30')
99
KRW-AXS : 구매전 매수 준비중
(148750.0, '2021-10-14 15:49:31')
100
KRW-STX : 구매전 매수 준비중
(2665.0, '2021-10-14 15:49:32')
101
0
KRW-BTC : 구매전 매수 준비중
(70438000.0, '2021-10-14 15:49:34')
1
KRW-ETH : 구매전 매수 준비중
(4418000.0, '2021-10-14 15:49:35')
2
KRW-NEO : 구매전 매수 준비중
(54860.0, '2021-10-14 15:49:36')
3
KRW-MTL : 구매전 매수 준비중
(4240.0, '2021-10-14 15:49:37')
4
KRW-LTC : 구매전 매수 준비중
(218500.0, '2021-10-14 15:49:38')
5
KRW-XRP : 구매전 매수 준비중
(1380.0, '2021-10-14 15:49:39')
6
KRW-ETC : 구매전 매수 준비중
(65010.0, '2021-10-14 15:49:40')
7
KRW-OMG : 구매전 매수 준비중
(17990.0, '2021-10-14 15:49:41')
8
KRW-SNT : 구매전 매수 준비중
(112.0,

KRW-RFR : 구매전 매수 준비중
(23.1, '2021-10-14 15:51:50')
37
KRW-CVC : 구매전 매수 준비중
(615.0, '2021-10-14 15:51:51')
38
KRW-IQ : 구매전 매수 준비중
(26.4, '2021-10-14 15:51:52')
39
KRW-IOTA : 구매전 매수 준비중
(1670.0, '2021-10-14 15:51:53')
40
KRW-MFT : 구매전 매수 준비중
(17.2, '2021-10-14 15:51:54')
41
KRW-ONG : 구매전 매수 준비중
(1515.0, '2021-10-14 15:51:55')
42
KRW-GAS : 구매전 매수 준비중
(11420.0, '2021-10-14 15:51:56')
43
KRW-UPP : 구매전 매수 준비중
(262.0, '2021-10-14 15:51:57')
44
KRW-ELF : 구매전 매수 준비중
(807.0, '2021-10-14 15:51:57')
45
KRW-KNC : 구매전 매수 준비중
(2115.0, '2021-10-14 15:51:58')
46
KRW-BSV : 구매전 매수 준비중
(211850.0, '2021-10-14 15:51:59')
47
KRW-THETA : 구매전 매수 준비중
(7425.0, '2021-10-14 15:52:00')
48
KRW-QKC : 구매전 매수 준비중
(45.0, '2021-10-14 15:52:01')
49
KRW-BTT : 구매전 매수 준비중
(4.6, '2021-10-14 15:52:02')
50
KRW-MOC : 구매전 매수 준비중
(212.0, '2021-10-14 15:52:03')
51
KRW-ENJ : 구매전 매수 준비중
(1935.0, '2021-10-14 15:52:04')
52
KRW-TFUEL : 구매전 매수 준비중
(356.0, '2021-10-14 15:52:06')
53
KRW-MANA : 구매전 매수 준비중
(943.0, '2021-10-14 15:52:07')
54
K

82
KRW-DOT : 구매전 매수 준비중
(50160.0, '2021-10-14 15:54:17')
83
KRW-SRM : 구매전 매수 준비중
(9290.0, '2021-10-14 15:54:18')
84
KRW-MVL : 구매전 매수 준비중
(20.1, '2021-10-14 15:54:19')
85
KRW-STRAX : 구매전 매수 준비중
(2785.0, '2021-10-14 15:54:20')
86
KRW-AQT : 구매전 매수 준비중
(5100.0, '2021-10-14 15:54:21')
87
KRW-GLM : 구매전 매수 준비중
(671.0, '2021-10-14 15:54:22')
88
KRW-SSX : 구매전 매수 준비중
(89.1, '2021-10-14 15:54:23')
89
KRW-META : 구매전 매수 준비중
(142.0, '2021-10-14 15:54:24')
90
KRW-FCT2 : 구매전 매수 준비중
(242.0, '2021-10-14 15:54:25')
91
KRW-CBK : 구매전 매수 준비중
(5640.0, '2021-10-14 15:54:26')
92
KRW-SAND : 구매전 매수 준비중
(902.0, '2021-10-14 15:54:27')
93
KRW-HUM : 구매전 매수 준비중
(251.0, '2021-10-14 15:54:28')
94
KRW-DOGE : 구매전 매수 준비중
(287.0, '2021-10-14 15:54:29')
95
KRW-STRK : 구매전 매수 준비중
(68000.0, '2021-10-14 15:54:30')
96
KRW-PUNDIX : 구매전 매수 준비중
(2040.0, '2021-10-14 15:54:31')
97
KRW-FLOW : 구매전 매수 준비중
(22720.0, '2021-10-14 15:54:32')
98
KRW-DAWN : 구매전 매수 준비중
(4215.0, '2021-10-14 15:54:33')
99
KRW-AXS : 구매전 매수 준비중
(148450.0, '2021-10

...

54
KRW-ANKR : 구매전 매수 준비중
(108.0, '2021-10-14 16:08:48')
55
KRW-AERGO : 구매전 매수 준비중
(406.0, '2021-10-14 16:08:49')
56
KRW-ATOM : 구매전 매수 준비중
(41640.0, '2021-10-14 16:08:50')
57
KRW-TT : 구매전 매수 준비중
(18.1, '2021-10-14 16:08:51')
58
KRW-CRE : 구매전 매수 준비중
(14.9, '2021-10-14 16:08:52')
59
KRW-MBL : 구매전 매수 준비중
(13.9, '2021-10-14 16:08:53')
60
KRW-WAXP : 구매전 매수 준비중
(350.0, '2021-10-14 16:08:54')
61
KRW-HBAR : 구매전 매수 준비중
(442.0, '2021-10-14 16:08:55')
62
KRW-MED : 구매전 매수 준비중
(90.2, '2021-10-14 16:08:56')
63
KRW-MLK : 구매전 매수 준비중
(1435.0, '2021-10-14 16:08:57')
64
KRW-STPT : 구매전 매수 준비중
(238.0, '2021-10-14 16:08:58')
65
KRW-ORBS : 구매전 매수 준비중
(149.0, '2021-10-14 16:08:59')
66
KRW-VET : 구매전 매수 준비중
(147.0, '2021-10-14 16:09:00')
67
KRW-CHZ : 구매전 매수 준비중
(399.0, '2021-10-14 16:09:01')
68
KRW-STMX : 구매전 매수 준비중
(40.3, '2021-10-14 16:09:02')
69
KRW-DKA : 구매전 매수 준비중
(184.0, '2021-10-14 16:09:03')
70
KRW-HIVE : 구매전 매수 준비중
(1050.0, '2021-10-14 16:09:04')
71
KRW-KAVA : 구매전 매수 준비중
(7325.0, '2021-10-14 16:09:05')


100
KRW-STX : 구매전 매수 준비중
(2660.0, '2021-10-14 16:11:14')
101
0
KRW-BTC : 구매전 매수 준비중
(70344000.0, '2021-10-14 16:11:16')
1
KRW-ETH : 구매전 매수 준비중
(4419000.0, '2021-10-14 16:11:17')
2
KRW-NEO : 구매전 매수 준비중
(54730.0, '2021-10-14 16:11:18')
3
KRW-MTL : 구매전 매수 준비중
(4250.0, '2021-10-14 16:11:19')
4
KRW-LTC : 구매전 매수 준비중
(218850.0, '2021-10-14 16:11:20')
5
KRW-XRP : 구매전 매수 준비중
(1380.0, '2021-10-14 16:11:21')
6
KRW-ETC : 구매전 매수 준비중
(65150.0, '2021-10-14 16:11:22')
7
KRW-OMG : 구매전 매수 준비중
(18020.0, '2021-10-14 16:11:23')
8
KRW-SNT : 구매전 매수 준비중
(113.0, '2021-10-14 16:11:24')
9
KRW-WAVES : 구매전 매수 준비중
(40550.0, '2021-10-14 16:11:25')
10
KRW-XEM : 구매전 매수 준비중
(215.0, '2021-10-14 16:11:26')
11
KRW-QTUM : 구매전 매수 준비중
(16160.0, '2021-10-14 16:11:27')
12
KRW-LSK : 구매전 매수 준비중
(5015.0, '2021-10-14 16:11:28')
13
KRW-STEEM : 구매전 매수 준비중
(774.0, '2021-10-14 16:11:29')
14
KRW-XLM : 구매전 매수 준비중
(456.0, '2021-10-14 16:11:30')
15
KRW-ARDR : 구매전 매수 준비중
(423.0, '2021-10-14 16:11:31')
16
KRW-ARK : 구매전 매수 준비중
(2670.0, '2021

45
KRW-KNC : 구매전 매수 준비중
(2120.0, '2021-10-14 16:13:38')
46
KRW-BSV : 구매전 매수 준비중
(211250.0, '2021-10-14 16:13:39')
47
KRW-THETA : 구매전 매수 준비중
(7430.0, '2021-10-14 16:13:40')
48
KRW-QKC : 구매전 매수 준비중
(45.0, '2021-10-14 16:13:41')
49
KRW-BTT : 구매전 매수 준비중
(4.59, '2021-10-14 16:13:42')
50
KRW-MOC : 구매전 매수 준비중
(211.0, '2021-10-14 16:13:43')
51
KRW-ENJ : 구매전 매수 준비중
(1925.0, '2021-10-14 16:13:44')
52
KRW-TFUEL : 구매전 매수 준비중
(356.0, '2021-10-14 16:13:45')
53
KRW-MANA : 구매전 매수 준비중
(943.0, '2021-10-14 16:13:46')
54
KRW-ANKR : 구매전 매수 준비중
(108.0, '2021-10-14 16:13:47')
55
KRW-AERGO : 구매전 매수 준비중
(406.0, '2021-10-14 16:13:48')
56
KRW-ATOM : 구매전 매수 준비중
(41600.0, '2021-10-14 16:13:49')
57
KRW-TT : 구매전 매수 준비중
(18.0, '2021-10-14 16:13:50')
58
KRW-CRE : 구매전 매수 준비중
(14.8, '2021-10-14 16:13:51')
59
KRW-MBL : 구매전 매수 준비중
(13.9, '2021-10-14 16:13:51')
60
KRW-WAXP : 구매전 매수 준비중
(349.0, '2021-10-14 16:13:52')
61
KRW-HBAR : 구매전 매수 준비중
(442.0, '2021-10-14 16:13:53')
62
KRW-MED : 구매전 매수 준비중
(89.9, '2021-10-14 16:13:54'

91
KRW-CBK : 구매전 매수 준비중
(5730.0, '2021-10-14 16:16:08')
92
KRW-SAND : 구매전 매수 준비중
(904.0, '2021-10-14 16:16:09')
93
KRW-HUM : 구매전 매수 준비중
(250.0, '2021-10-14 16:16:10')
94
KRW-DOGE : 구매전 매수 준비중
(288.0, '2021-10-14 16:16:11')
95
KRW-STRK : 구매전 매수 준비중
(67910.0, '2021-10-14 16:16:12')
96
KRW-PUNDIX : 구매전 매수 준비중
(2035.0, '2021-10-14 16:16:13')
97
KRW-FLOW : 구매전 매수 준비중
(22590.0, '2021-10-14 16:16:14')
98
KRW-DAWN : 구매전 매수 준비중
(4245.0, '2021-10-14 16:16:15')
99
KRW-AXS : 구매전 매수 준비중
(148800.0, '2021-10-14 16:16:16')
100
KRW-STX : 구매전 매수 준비중
(2665.0, '2021-10-14 16:16:17')
101
0
KRW-BTC : 구매전 매수 준비중
(70263000.0, '2021-10-14 16:16:19')
1
KRW-ETH : 구매전 매수 준비중
(4415000.0, '2021-10-14 16:16:19')
2
KRW-NEO : 구매전 매수 준비중
(54710.0, '2021-10-14 16:16:20')
3
KRW-MTL : 구매전 매수 준비중
(4240.0, '2021-10-14 16:16:21')
4
KRW-LTC : 구매전 매수 준비중
(218500.0, '2021-10-14 16:16:22')
5
KRW-XRP : 구매전 매수 준비중
(1380.0, '2021-10-14 16:16:23')
6
KRW-ETC : 구매전 매수 준비중
(65070.0, '2021-10-14 16:16:24')
7
KRW-OMG : 구매전 매수 준비중
(18010.

35
KRW-IOST : 구매전 매수 준비중
(66.4, '2021-10-14 16:18:34')
36
KRW-RFR : 구매전 매수 준비중
(22.7, '2021-10-14 16:18:35')
37
KRW-CVC : 구매전 매수 준비중
(618.0, '2021-10-14 16:18:36')
38
KRW-IQ : 구매전 매수 준비중
(26.3, '2021-10-14 16:18:37')
39
KRW-IOTA : 구매전 매수 준비중
(1670.0, '2021-10-14 16:18:38')
40
KRW-MFT : 구매전 매수 준비중
(17.1, '2021-10-14 16:18:39')
41
KRW-ONG : 구매전 매수 준비중
(1515.0, '2021-10-14 16:18:40')
42
KRW-GAS : 구매전 매수 준비중
(11430.0, '2021-10-14 16:18:41')
43
KRW-UPP : 구매전 매수 준비중
(262.0, '2021-10-14 16:18:42')
44
KRW-ELF : 구매전 매수 준비중
(805.0, '2021-10-14 16:18:43')
45
KRW-KNC : 구매전 매수 준비중
(2120.0, '2021-10-14 16:18:44')
46
KRW-BSV : 구매전 매수 준비중
(211050.0, '2021-10-14 16:18:45')
47
KRW-THETA : 구매전 매수 준비중
(7430.0, '2021-10-14 16:18:46')
48
KRW-QKC : 구매전 매수 준비중
(44.7, '2021-10-14 16:18:47')
49
KRW-BTT : 구매전 매수 준비중
(4.59, '2021-10-14 16:18:48')
50
KRW-MOC : 구매전 매수 준비중
(212.0, '2021-10-14 16:18:49')
51
KRW-ENJ : 구매전 매수 준비중
(1925.0, '2021-10-14 16:18:50')
52
KRW-TFUEL : 구매전 매수 준비중
(355.0, '2021-10-14 16:18:51')
